In [ ]:
# Install the specified version of the Hugging Face Hub package
!pip install -U huggingface-hub==0.27.1  

import wandb  

# Handling Kaggle Secrets  
from kaggle_secrets import UserSecretsClient  
user_secrets = UserSecretsClient()  

# Retrieve API tokens from Kaggle secrets  
hf_token = user_secrets.get_secret("HF_TOKEN")  
wb_token = user_secrets.get_secret("WANDB_TOKEN")  

# Login to Weights & Biases  
wandb.login(key=wb_token)  

# Login to Hugging Face Hub  
from huggingface_hub import login  
login(token=hf_token)  


In [ ]:
# !pip install -U huggingface-hub==0.27.1 transformers==4.46.1 datasets==3.1.0 torch==2.5.1+cu121 bitsandbytes==0.45.0

import os
import json
from datasets import load_dataset
import torch

# Set the working directory
os.chdir('/kaggle/working/')
os.system('rm -rf LLaMA-Factory')  # Remove any existing LLaMA-Factory directory
os.system('git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git')  # Clone the repository
os.chdir('LLaMA-Factory')

# Install required dependencies
!pip install -q torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -q -y jax  # Uninstall JAX if present
!pip install -q -e .[torch,bitsandbytes,liger-kernel]  # Install LLaMA-Factory dependencies

# Verify CUDA availability
try:
    assert torch.cuda.is_available() is True
except AssertionError:
    print("Please set up a GPU before using LLaMA Factory")


# Hyperparameter Configuration

In [ ]:
stage = "dpo"  # Training stage: Direct Preference Optimization (DPO)
data_size = 20000  # Number of training samples
base_model = "Llama-3.2-1B-Instruct"  # Pretrained base model used for fine-tuning
dataset_name = "Muadil/dpo_dataset_train_openai_summary"  # Dataset path in Hugging Face Datasets
batch_size = 2  # Number of samples per training batch
epoch_size = 3  # Number of training epochs


In [ ]:
# Loading the Hugging Face dataset
dataset = load_dataset(dataset_name)

# Selecting the first x data points
resized_dataset = dataset["train"].select(range(min(int(data_size * 1.25), len(dataset["train"]))))

# Splitting the dataset into 80% training and 20% evaluation
train_test_split = resized_dataset.train_test_split(test_size=0.2, seed=42)

# Extracting training and evaluation sets
train_dataset = train_test_split["train"]["prompt"]
eval_dataset = train_test_split["test"]["prompt"]

# Saving datasets to JSON files
train_output_file = f"data/{stage}_dataset_{data_size}.json"
with open(train_output_file, "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, indent=2, ensure_ascii=False)

eval_output_file = f"data/{stage}_eval_dataset_{data_size}.json"
with open(eval_output_file, "w", encoding="utf-8") as f:
    json.dump(eval_dataset, f, indent=2, ensure_ascii=False)

import json

# Loading the dataset information file
with open("data/dataset_info.json", "r", encoding="utf-8") as f:
    dataset_info = json.load(f)

# Creating new dataset entries
new_data = {
    f"{stage}_dataset_{data_size}": {
        "file_name": f"{stage}_dataset_{data_size}.json",
        "ranking": True,
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "chosen": "chosen",
            "rejected": "rejected",
        },
    }
}

new_eval_data = {
    f"{stage}_eval_dataset_{data_size}": {
        "file_name": f"{stage}_eval_dataset_{data_size}.json",
        "ranking": True,
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "chosen": "chosen",
            "rejected": "rejected",
        },
    }
}

# Updating dataset information with new entries
dataset_info.update(new_data)
dataset_info.update(new_eval_data)

# Saving the updated dataset information
with open("data/dataset_info.json", "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, indent=2, ensure_ascii=False)

print("New dataset entries have been successfully added and saved.")

print(f"The first {data_size} data points have been successfully saved to {train_output_file} and {eval_output_file}.")


In [ ]:

import json
args = dict(
  quantization_bit=4, #nf4 to do quantization, default is None so it does not quantize.
  quantization_method="bitsandbytes", # we enter in which format to quantize. bitsandbytes, hqq, eetq, etc.
  stage=stage,
  do_train=True,
  pref_loss = "sigmoid",
  model_name_or_path=f"meta-llama/{base_model}", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset=f"{stage}_dataset_{data_size}",             # use alpaca and identity datasets
  #eval_dataset = f"{stage}_eval_dataset_{data_size}",
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                   # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                  # the path to save LoRA adapters
  per_device_train_batch_size=batch_size,               # the batch size
  gradient_accumulation_steps=4,               # the gradient accumulation steps
  lr_scheduler_type="cosine",                 # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=1000,                      # save checkpoint every 1000 steps
  learning_rate=5e-5,                     # the learning rate
  num_train_epochs=epoch_size,                    # the epochs of training
  max_samples=500,                      # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                   # use LoRA+ algorithm with lambda=16.0
  fp16=True,                         # use float16 mixed precision training
  enable_liger_kernel=True,                   # use liger kernel for efficient training
  report_to="wandb",
  run_name="username/repository",
  #load_best_model_at_end=True,  # Ensure the best model is loaded at the end
  metric_for_best_model="eval_loss",  # Metric to monitor for the best model
  greater_is_better=False,
  #evaluation_strategy="steps",
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)
# Start the training process
!llamafactory-cli train train_llama3.json



In [ ]:
# Saving and exporting the model
export_args = {
    "model_name_or_path": f"meta-llama/{base_model}",  # Original model
    "adapter_name_or_path": "llama3_lora",  # Saved LoRA adapters
    "template": "llama3",  # Same template
    "finetuning_type": "lora",  # Using the same LoRA method
    "export_dir": "llama3_lora_merged",  # Directory to store the merged model
    "export_size": 5,  # Model file size (GB)
    "export_device": "cpu",  # Device for exporting
    "export_hub_model_id": "username/repository",  # Hugging Face model ID
}

# Save export arguments to a JSON file
with open("merge_llama3.json", "w", encoding="utf-8") as f:
    json.dump(export_args, f, indent=2)

# Export the model and upload it to Hugging Face
!llamafactory-cli export merge_llama3.json
